In [1]:
%env CUDA_VISIBLE_DEVICES=0
import torch

device=["cpu", "cuda"][torch.cuda.is_available()]
print(device)

env: CUDA_VISIBLE_DEVICES=0
cuda


In [2]:
import json
import os.path as osp

with open("./conf/pretrain_pcap.json") as f:
    config=json.load(f)

pt="./work_dir/pt_source/"
mod="Salesforce/blip-image-captioning-base"


In [3]:
import evaluate_model
evaluate_model.tqdm_module(True)
from pretrain import load_model

model, processor, load_info=load_model(config, pt, device)

-------initializing base model from ./work_dir/pt_source/ --------
Image size:224

-------loading ckpt for pretrain-------


In [ ]:
model.get_pfx_status()

{'ViT_prefix': True, 'Text_dec_prefix': False}

In [ ]:
load_info

In [6]:
# load cached result
def load_cached_result(json_filename:str):
    if osp.exists(json_filename):
        f=open(json_filename, "r+")
        cached_results=json.load(f)
        f.close()
    else:
        cached_results=[]
    
    return cached_results

def dump_result(results, json_fname:str):
    with open(json_fname, "w+") as f:
        json.dump(results, f, indent=4)

In [7]:
import os, re

# return all ckpt files from 
def find_ckpt_files(dir, pattern):
    files=os.listdir(dir)
    ckpts=list(filter(lambda x: re.match(pattern, x) is not None, files))
    # extract epoch numbers from valid ckpt list
    epoches=list(map(lambda x:int(re.findall(pattern, x)[0]), ckpts))
    # concat address
    ckpts=list(map(lambda x:os.path.join(dir, x), ckpts))

    zipped_ckpts=list(zip(epoches,ckpts))

    return sorted(zipped_ckpts, key=lambda x:x[0])

In [8]:
# visualize the results in pd
import pandas as pd

def dump_dataframe(results, index="epoch"):
    df=pd.read_json(json.dumps(results)).set_index(index)
    df=df.sort_index()

    return df

## test_model_output

In [9]:
from transformers import BlipConfig, BlipProcessor
from model import ConCapPretrainedModel

pt="./work_dir/pt_source/"

model_conf=BlipConfig.from_pretrained(pt)
# test_mo=BlipModel.from_pretrained(mod)
test_itc=model
preprocessor=BlipProcessor.from_pretrained(pt)

### for test new model

In [3]:
from transformers import BlipConfig, BlipProcessor
from model.concap_model import ConCapModel

pt="./work_dir/pt_source_14m/"

model_conf=BlipConfig.from_pretrained(pt)
# test_mo=BlipModel.from_pretrained(mod)
# test_itc=ConCapModelNew(model_conf, config["text_model"])
test_itc=ConCapModel.from_pretrained(pt, config["text_model"])
preprocessor=BlipProcessor.from_pretrained(pt)

Some weights of the model checkpoint at ./work_dir/pt_source_14m/ were not used when initializing ConCapModel: ['text_encoder.encoder.layer.8.output.LayerNorm.bias', 'text_encoder.encoder.layer.9.crossattention.self.query.bias', 'text_encoder.encoder.layer.7.output.LayerNorm.weight', 'text_encoder.encoder.layer.5.attention.output.LayerNorm.bias', 'text_encoder.encoder.layer.9.crossattention.self.key.bias', 'text_encoder.encoder.layer.11.output.dense.weight', 'text_encoder.encoder.layer.1.attention.output.dense.bias', 'text_encoder.encoder.layer.1.output.LayerNorm.weight', 'text_encoder.encoder.layer.11.output.dense.bias', 'text_encoder.encoder.layer.3.attention.self.key.bias', 'text_encoder.encoder.layer.9.output.dense.weight', 'text_encoder.encoder.layer.1.attention.self.value.bias', 'text_encoder.encoder.layer.4.attention.self.value.bias', 'text_encoder.encoder.layer.4.attention.output.LayerNorm.weight', 'text_encoder.encoder.layer.1.crossattention.output.LayerNorm.weight', 'text_enc

In [4]:
import data

batch=2
test_ds=data.build_pcap_dataset(config, preprocessor, device, "train")
test_dl=data.build_dataloader(test_ds,batch,0,False)


-------dataset loading-------


Using custom data configuration default-4e293306064a41a3
Found cached dataset json (/home/lab-gao.zhenpeng/.cache/huggingface/datasets/json/default-4e293306064a41a3/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)



Load train split from /148Dataset/data-gao.zhenpeng/PCap/personality_captions/train.json
Length: 186800
-------dataset load done-------


In [5]:
dl_item=dict(next(iter(test_dl)))
dl_item["labels"]=dl_item["input_ids"]

for k,item in dl_item.items():
    print(k, item.shape)


pixel_values torch.Size([2, 3, 224, 224])
input_ids torch.Size([2, 90])
attention_mask torch.Size([2, 90])
prefix_ids torch.Size([2, 16])
labels torch.Size([2, 90])


In [6]:
from model.utils.generation import generate_nucleus

@torch.no_grad()
def generate_test(model, pixel_values, preprocessor, topk=10, topp=0.9, temp=0.75):
    device=model.device
    batch_size = pixel_values.shape[0]
    vision_outputs = model.vision_model(
        pixel_values=pixel_values
    )

    image_embeds=vision_outputs[0]
    image_attention_mask = torch.ones(image_embeds.size()[:-1], 
                                        dtype=torch.long).to(device)
    
    # dummy pfx
    prefix_embs=torch.ones(batch_size, 1, model.config.text_config.hidden_size).to(device)
    prefix_embs.fill_(model.config.text_config.pad_token_id)
    # dummy pfx id
    prefix_ids=preprocessor.tokenizer(["a picture of"])["input_ids"][0][1:-1]
    prefix_ids=torch.tensor(prefix_ids, dtype=torch.long, device=device).repeat(batch,1)
    results=model.generate(pixel_values, prefix_ids, num_beams=3, max_length=30)
    results=preprocessor.batch_decode(results)

    return results


In [10]:
preprocessor.tokenizer(["a photo of"])["input_ids"][0][1:-1]

[1037, 6302, 1997]

In [ ]:
generate_test(test_itc, dl_item["pixel_values"], preprocessor)

In [9]:
out=test_itc(**dl_item)

In [ ]:
out

In [ ]:
gen=test_itc.generate(pixel_values=dl_item["pixel_values"],
                      prefix_ids=dl_item["prefix_ids"],
                      max_length=30)

In [10]:
out["dec_logits"].shape

torch.Size([2, 16, 30524])

### 

In [10]:
print(id(test_itc.text_encoder.embeddings.word_embeddings.weight))
print(id(test_itc.text_decoder.bert.embeddings.word_embeddings.weight))

139665905517808
139665905517808


In [11]:
import data

batch=2
test_ds=data.build_pcap_dataset(config, preprocessor, device, "pretrain")
test_dl=data.build_dataloader(test_ds,batch,0,False)


-------dataset loading-------


Using custom data configuration default-4e293306064a41a3
Found cached dataset json (/home/lab-gao.zhenpeng/.cache/huggingface/datasets/json/default-4e293306064a41a3/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)



Load pretrain split from /148Dataset/data-gao.zhenpeng/PCap/personality_captions/train.json
Length: 186800
-------dataset load done-------


In [12]:
def merge_negative(entity, device):
    pixel_values=entity.pop("pixel_values").to(device)
    bsz=pixel_values.shape[0]
    merges={}
    # add label first
    
    for merge_keys in ["input_ids", "attention_mask", "prefix_ids"]:
        gnd_truth=entity.pop(merge_keys)
        fake=entity.pop("neg_{}".format(merge_keys))
        merges[merge_keys]=torch.cat([gnd_truth, fake], dim=0).to(device)
    
    labels=merges["input_ids"].clone()[0:bsz, :]
    
    return dict(pixel_values=pixel_values, labels=labels, **merges)

In [13]:
dl_item=next(iter(test_dl))
dl_merged=merge_negative(dl_item, device)

for k,item in dl_merged.items():
    print(k, item.shape)

pixel_values torch.Size([2, 3, 224, 224])
labels torch.Size([2, 90])
input_ids torch.Size([4, 90])
attention_mask torch.Size([4, 90])
prefix_ids torch.Size([4, 16])


In [14]:
test_itc.prefix_decoder_(False)
test_itc.get_pfx_status()

{'ViT_prefix': True, 'Text_dec_prefix': False}

In [27]:
test_itc.shrink_sv=True
out=test_itc(**dl_merged)


In [28]:
out

{'loss_itc': tensor(1.1126, device='cuda:0', grad_fn=<DivBackward0>),
 'loss_lm': tensor(11.1471, device='cuda:0', grad_fn=<AddBackward0>),
 'img_embeds': tensor([[[ 0.3283,  0.0837,  0.2164,  ...,  0.3975, -0.0954,  0.1929],
          [-0.2062,  1.9501,  0.8359,  ..., -0.4714,  0.0673,  0.6110],
          [ 0.7568,  1.3493,  0.0706,  ..., -0.1209, -0.4022,  0.3815],
          ...,
          [-0.3864,  0.7855,  0.2927,  ..., -0.1600, -0.0219,  0.3021],
          [ 0.3155,  0.6456,  0.1878,  ..., -0.0556,  0.1236,  0.0337],
          [ 0.0796,  0.8557, -0.3193,  ..., -0.7421, -0.2081,  0.5640]],
 
         [[-0.4144, -0.1031, -0.0307,  ..., -0.2658,  0.1297,  0.4910],
          [ 0.2670,  0.0143, -1.7533,  ...,  0.3614, -0.0510, -0.8377],
          [-1.6325,  1.1176,  0.2287,  ...,  0.1843, -0.4375, -0.0537],
          ...,
          [ 0.1410,  0.4060, -0.7311,  ..., -0.6398,  1.2749, -0.1180],
          [-0.2414,  0.5608,  0.6896,  ..., -0.5657,  0.3225,  0.0545],
          [ 0.2522,  

In [15]:
gen=test_itc.generate(
    pixel_values=dl_merged["pixel_values"],
    prefix_ids=dl_merged["prefix_ids"][batch:],
    max_length=30
)

In [16]:
processor.batch_decode(gen, skip_special_tokens=True)

['a red red red red red red red red red red red a red red red red red red red red red red red red red red red red',
 'black black black black black black a a a a a a a a black a black a black a black a black a black a black a black']

## test inference

In [9]:
def img_hash_to_addr(x, img_addr, img_attr):
    x["images"]=osp.join(img_addr, x.pop("image_hash")+img_attr)

    return x

img_addr=config["dataset"]["img_path"]
img_attr=config["dataset"]["img_attr"]

with open("./misc/inference_src.json") as f:
    inference_list=json.load(f)

inference_list=[img_hash_to_addr(x, img_addr, img_attr) for x in inference_list]

In [28]:
# load ckpt
epoch=8
inference_ckpt="./work_dir/pretrain/pretrain_epoch{}.pth".format(epoch)
state_dict=torch.load(inference_ckpt)
missing_keys, unexp_keys=model.load_state_dict(state_dict, strict=False)

In [29]:
model.get_pfx_status()

{'ViT_prefix': True, 'Text_dec_prefix': False}

In [30]:
inf_r=evaluate_model.inference(model, config, processor, device, inference_list)

------Inference start-------
------Inference end-------


In [31]:
inf_r

[{'personality': 'Stupid',
  'images': '/148Dataset/data-gao.zhenpeng/PCap/yfcc_images/15cdd44cf6d73b8f0b64352372a91c1.jpg',
  'comment_model': 'i would love to take this car for a spin, it would be so much fun',
  'comment_gndtruth': 'why do cones even exist'},
 {'personality': 'Opinionated',
  'images': '/148Dataset/data-gao.zhenpeng/PCap/yfcc_images/2ed2094a52eb3579f4ef7c5e501db5.jpg',
  'comment_model': 'i love the way her dress matches the wall',
  'comment_gndtruth': 'I would never wear that top with those pants.'},
 {'personality': 'Deep',
  'images': '/148Dataset/data-gao.zhenpeng/PCap/yfcc_images/f07589c35e378e3043fd5f642513bb64.jpg',
  'comment_model': "i'm not even sure what kind of bug this is",
  'comment_gndtruth': 'This was found in a glacier.'},
 {'personality': 'Mystical',
  'images': '/148Dataset/data-gao.zhenpeng/PCap/yfcc_images/ac75942b5d6261354442cf502a2bb7dd.jpg',
  'comment_model': 'i would love to see this in real life, but i have to look at a bunch of these pi

## test_automated_LM

In [14]:
lm_json="./work_dir/pretrain/lm_230806b.json"
lm_results=load_cached_result(lm_json)

In [15]:
tested_epoches=[x["epoch"] for x in lm_results]
ckpt_dir="./work_dir/pretrain/"
fname_pattern=re.compile(r"^pretrain_epoch(\d+?)\.pth$")
ckpt_lists=find_ckpt_files(ckpt_dir, fname_pattern)
un_ckpt_lists=list(filter(lambda x:x[0] not in tested_epoches, ckpt_lists))

un_ckpt_lists

[(2, './work_dir/pretrain/pretrain_epoch2.pth'),
 (4, './work_dir/pretrain/pretrain_epoch4.pth'),
 (6, './work_dir/pretrain/pretrain_epoch6.pth'),
 (8, './work_dir/pretrain/pretrain_epoch8.pth'),
 (10, './work_dir/pretrain/pretrain_epoch10.pth')]

In [16]:
for epoch, fname in un_ckpt_lists:
    print("\nEvaluating epoch {}".format(epoch))

    state_dict=torch.load(fname)
    missing_keys, unexp_keys=model.load_state_dict(state_dict, strict=False)
    
    lm_result=evaluate_model.evaluate_lm(model, config, processor, device)
    print(lm_result)
    lm_results.append({"epoch": epoch, **lm_result})
    



Evaluating epoch 2

------Enter evaluation------
Using pycocoeval metric currently. 

-------dataset loading-------

Load test split from /148Dataset/data-gao.zhenpeng/PCap/personality_captions/test.json
Length: 9996
-------dataset load done-------


Processing test set:   0%|          | 0/1250 [00:00<?, ?it/s]

PTBTokenizer tokenized 130267 tokens at 902555.02 tokens per second.
Aug 07, 2023 1:57:41 AM edu.stanford.nlp.process.PTBLexer next
PTBTokenizer tokenized 529941 tokens at 1826223.87 tokens per second.


Evaluating ['BLEU_1', 'BLEU_4']...
{'testlen': 120272, 'reflen': 109450, 'guess': [120272, 110276, 100280, 90284], 'correct': [36957, 4330, 598, 158]}
ratio: 1.0988761991776967
Evaluating ROUGE_L...
Evaluating CIDEr...
Evaluating SPICE...


Parsing reference captions
Parsing test captions


...

Processing test set:   0%|          | 0/1250 [00:00<?, ?it/s]

PTBTokenizer tokenized 112893 tokens at 678552.72 tokens per second.
Aug 07, 2023 2:20:12 AM edu.stanford.nlp.process.PTBLexer next
PTBTokenizer tokenized 529941 tokens at 1919012.10 tokens per second.


Evaluating ['BLEU_1', 'BLEU_4']...
{'testlen': 102898, 'reflen': 94573, 'guess': [102898, 92902, 82906, 72910], 'correct': [33232, 4193, 800, 211]}
ratio: 1.0880272382180844
Evaluating ROUGE_L...
Evaluating CIDEr...
Evaluating SPICE...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.9 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.5

SPICE evaluation took: 32.83 s

------Finished evaluation------
{'BLEU_1': 32.296, 'BLEU_4': 2.526, 'ROUGE_L': 21.247, 'CIDEr': 7.967, 'SPICE': 3.784}


In [17]:
lm_results

[{'epoch': 2,
  'BLEU_1': 30.728,
  'BLEU_4': 1.884,
  'ROUGE_L': 21.475,
  'CIDEr': 8.056,
  'SPICE': 4.49},
 {'epoch': 4,
  'BLEU_1': 31.786,
  'BLEU_4': 3.058,
  'ROUGE_L': 20.954,
  'CIDEr': 8.435,
  'SPICE': 2.691},
 {'epoch': 6,
  'BLEU_1': 36.566,
  'BLEU_4': 3.435,
  'ROUGE_L': 23.107,
  'CIDEr': 9.34,
  'SPICE': 3.747},
 {'epoch': 8,
  'BLEU_1': 33.155,
  'BLEU_4': 2.578,
  'ROUGE_L': 21.557,
  'CIDEr': 8.221,
  'SPICE': 3.724},
 {'epoch': 10,
  'BLEU_1': 32.296,
  'BLEU_4': 2.526,
  'ROUGE_L': 21.247,
  'CIDEr': 7.967,
  'SPICE': 3.784}]

In [18]:
dump_result(lm_results, lm_json)

## test ITM

In [11]:
from math import ceil
from data import build_dataloader, build_pcap_dataset
eval_batch=16
eval_workers=4
eval_ds=build_pcap_dataset(config, processor, device, "val")
eval_dl=build_dataloader(eval_ds, batch=eval_batch, num_workers=eval_workers, dist_training=False)

total_steps=ceil(len(eval_ds)/eval_batch)



-------dataset loading-------

Load val split from /148Dataset/data-gao.zhenpeng/PCap/personality_captions/val.json
Length: 4999
-------dataset load done-------


In [12]:
itm_json="./work_dir/pretrain/itm_230806.json"
itm_results=load_cached_result(itm_json)

In [13]:
tested_epoches=[x["epoch"] for x in itm_results]

ckpt_dir="./work_dir/pretrain/"
fname_pattern=re.compile(r"^pretrain_epoch(\d+?)\.pth$")
ckpt_lists=find_ckpt_files(ckpt_dir, fname_pattern)
un_ckpt_lists=list(filter(lambda x:x[0] not in tested_epoches, ckpt_lists))

un_ckpt_lists

[(1, './work_dir/pretrain/pretrain_epoch1.pth'),
 (2, './work_dir/pretrain/pretrain_epoch2.pth'),
 (3, './work_dir/pretrain/pretrain_epoch3.pth'),
 (4, './work_dir/pretrain/pretrain_epoch4.pth'),
 (5, './work_dir/pretrain/pretrain_epoch5.pth'),
 (6, './work_dir/pretrain/pretrain_epoch6.pth')]

In [14]:
for epoch, fname in un_ckpt_lists:
    print("\nEvaluating epoch {} of ITM".format(epoch))

    state_dict=torch.load(fname)
    missing_keys, unexp_keys=model.load_state_dict(state_dict, strict=False)
    
    itm_result=evaluate_model.evaluate_itm(model, eval_dl, device=device, 
                                           total_steps=total_steps, epoch=epoch)
    print(itm_result[1])
    itm_results.append(itm_result[1])


Evaluating epoch 1 of ITM


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 1
ITM loss: 0.8491393327713013
ITM accuracy: 73.17%
{'epoch': 1, 'accuracy': 73.1746, 'itm_loss': 0.8491}

Evaluating epoch 2 of ITM


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 2
ITM loss: 0.8590194582939148
ITM accuracy: 72.83%
{'epoch': 2, 'accuracy': 72.8346, 'itm_loss': 0.859}

Evaluating epoch 3 of ITM


...

Epoch: 4
ITM loss: 0.9726016521453857
ITM accuracy: 35.55%
{'epoch': 4, 'accuracy': 35.5471, 'itm_loss': 0.9726}

Evaluating epoch 5 of ITM


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 5
ITM loss: 1.1176306009292603
ITM accuracy: 37.69%
{'epoch': 5, 'accuracy': 37.6875, 'itm_loss': 1.1176}

Evaluating epoch 6 of ITM


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 6
ITM loss: 1.2276833057403564
ITM accuracy: 39.29%
{'epoch': 6, 'accuracy': 39.2879, 'itm_loss': 1.2277}


In [15]:
itm_results

[{'epoch': 1, 'accuracy': 73.1746, 'itm_loss': 0.8491},
 {'epoch': 2, 'accuracy': 72.8346, 'itm_loss': 0.859},
 {'epoch': 3, 'accuracy': 55.8112, 'itm_loss': 0.8762},
 {'epoch': 4, 'accuracy': 35.5471, 'itm_loss': 0.9726},
 {'epoch': 5, 'accuracy': 37.6875, 'itm_loss': 1.1176},
 {'epoch': 6, 'accuracy': 39.2879, 'itm_loss': 1.2277}]

In [61]:
dump_result(itm_results, itm_json)

## test finetune inference

In [6]:
import json
import os.path as osp

with open("./conf/caption_pcap_224.json") as f:
    config=json.load(f)

pt=osp.join(config["work_dir"], "outputs")

In [ ]:
import evaluate_model
evaluate_model.tqdm_module(True)
from finetune_pfx_dec import load_model as load_finetune_model

model, processor, load_info=load_finetune_model(config, pt, device)

In [6]:
model.get_pfx_status()

{'ViT_prefix': False, 'Text_dec_prefix': True}

In [7]:
load_info

{'missing_keys': [],
 'unexpected_keys': [],
 'mismatched_keys': [],
 'error_msgs': []}

In [8]:
def img_hash_to_addr(x, img_addr, img_attr):
    x["images"]=osp.join(img_addr, x.pop("image_hash")+img_attr)

    return x

img_addr=config["dataset"]["img_path"]
img_attr=config["dataset"]["img_attr"]

with open("./misc/inference_src.json") as f:
    inference_list=json.load(f)

inference_list=[img_hash_to_addr(x, img_addr, img_attr) for x in inference_list]

In [10]:
# load ckpt
inference_ckpt=osp.join(config["work_dir"], "last.pth")
state_dict=torch.load(inference_ckpt)
missing_keys, unexp_keys=model.load_state_dict(state_dict, strict=False)

In [12]:
missing_keys, unexp_keys

([], [])

In [13]:
model.get_pfx_status()

{'ViT_prefix': False, 'Text_dec_prefix': True}

In [14]:
inf_r=evaluate_model.inference(model, config, processor, device, inference_list)

------Inference start-------
------Inference end-------


In [ ]:
inf_r